# Introduction to PyEZ

PyEZ is a collection of open source python modules, authored by Juniper that facilitate the automation of devices running JunOS.



In [ ]:
import jnpr.junos.exception 
from jnpr.junos import Device
from lxml import etree
from pprint import pprint


You authenticate with a standard username and password configured on the destination device(s).

```
netconf@JNCIA-DevOps-Test-SRX> show configuration system login user netconf 
uid 2007;
class super-user;
authentication {
    encrypted-password "$1$65GprHhA$QBnUF8WHyKb8v3V7VTBzO0"; ## SECRET-DATA
}
```


We start with the <i>Device</i> class we've imported from jnpr.junos.  This controls connecting and interfacing with JunOS devices.

We instantiate the class with the host we want to connect to and some authentication parameters.  Then we run the <i>open()</i> method from the instantiated object.  At this point a connection is established out to the JunOS device.



In [ ]:
UID = 'netconf'
PWD = 'netconf123'
ROUTER = '192.168.1.1'

dev = Device(host=ROUTER, password=PWD, user=UID, normalize=True) # connect via NETCONF by default
                                                                  # normalize trims whitespace from xml replies 

#dev = Device(host=ROUTER, password=PWD, user=UID, mode='telnet') # You can also connect via telnet
#dev = Device(host=ROUTER, password=PWD, user=UID, mode='serial', port='/dev/ttyUSB0') # or console
#dev = Device(host=ROUTER, user=UID)   # If you have SSH keys installed, you don't need to specify a password



dev.open()
print(dev.facts['hostname'])

<b>Facts</b> is a nested dictionary that is pulled together and returned by default after the initial connection.  It contains parameters describing the hardware and software state of the remote device.

As it's a dictionary, values can be specifically selected using their keys e.g. 
try: ```print(dev.facts['RE0']['up_time'])```

If the information within it is of no use to you, you can exclude it to reduce connect time:

```dev = Device(host=ROUTER, user=UID, get_facts=False)```

In [ ]:
pprint(dev.facts)

Once the connection is established we can get to the business of querying for the data we need.  To find the relevant RPC to use you can just look at the CLI:

```
netconf@JNCIA-DevOps-Test-SRX> show security flow session policy-id 1| display xml rpc    
<rpc-reply xmlns:junos="http://xml.juniper.net/junos/12.3X48/junos">
    <rpc>
        <get-flow-session-information>
                <policy-id>1</policy-id>
        </get-flow-session-information>
         </rpc>
        <cli>
            <banner></banner>
        </cli>
    </rpc-reply>

```

We need to convert the '-' to '_ '.  Python interprets '-' as the minus operator so tries to evaluate as an expression.

In [ ]:
security_flows = dev.rpc.get_flow_session_information(policy_id='1')


Now we have the XML data we can iterate over it and use it as we see fit.
We could use a full XPATH expression as outlined in Section 2 i.e.: ```xml_doc.xpath('xpath_expression')```

Alternatively you can use the simpler <i>.find()</i> or <i>.findall()</i> methods.  These return either the first match or all matches for basic search patterns. For example:

In [ ]:
print("{:15} {:15} {:15} {:15}".format('Src IP', 'Src Port', 'Dest IP', 'Dest Port'))

for elem in security_flows.findall('flow-session'):
    print("{:15} {:15} {:15} {:15}".format(
        elem.find('.//source-address').text,
        elem.find('.//source-port').text,
        elem.find('.//destination-address').text,
        elem.find('.//destination-port').text
        ))

Once you have all the data you need, you should close the session with the remote device

In [ ]:
dev.close()

An alternative to having to use <i>dev.open()</i> and <i>dev.close()</i> methods each time is to take advantage of the python 'with' construct which implicitly uses <i>.open()</i> immediately after object instantiation and uses <i>.close()</i> once execution in the stanza is complete.

In [ ]:
with Device(host=ROUTER, password=PWD, user=UID, normalize=True) as dev:
    print(dev.facts['hostname'])

